In [ ]:
# default_exp core

# handy-dict

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from copy import deepcopy

In [ ]:
# export
class ApplyKeyedException(Exception):
    
    def __init__(self, *args, path, value):
        self.path = path
        self.value = value

In [ ]:
# export
def apply_keyed(dictionary, keys, action, ignore_non_existing=True):
    dictionary_copy = deepcopy(dictionary)
    followed_path = []

    def apply_keyed_inplace(data, keys, path):
        if keys[0] not in data and ignore_non_existing:
            return

        if len(keys) == 1:
            try:
                data[keys[0]] = action(data[keys[0]])
            except Exception as excp:
                path.append(keys[0])
                raise ApplyKeyedException(
                    path=path, value=data.get(keys[0])
                ) from excp
        else:
            if isinstance(data[keys[0]], list):
                for idx, obj in enumerate(data[keys[0]]):
                    path.append(f"{keys[0]}[{idx}]")
                    apply_keyed_inplace(obj, keys[1:], path)
                    path.pop()
            else:
                path.append(keys[0])
                apply_keyed_inplace(data[keys[0]], keys[1:], path)
                path.pop()

    apply_keyed_inplace(dictionary_copy, keys, followed_path)
    return dictionary_copy

In [ ]:
def duplicate(text):
    return text + text

In [ ]:
test_dict = {'A': {'B': 'C'}}
expected = {'A': {'B': 'CC'}}
actual = apply_keyed(test_dict, ["A", "B"], duplicate)
assert actual == expected

In [ ]:
test_dict = {'A': [{'B': 'C'}]}
expected = {'A': [{'B': 'CC'}]}
actual = apply_keyed(test_dict, ["A", "B", ], duplicate)
assert actual == expected

In [ ]:
# export
def return_keyed(dictionary, keys, ignore_non_existing=True):
    return_values = []

    def apply_keyed_inplace(data, keys, return_values):
        if keys[0] not in data and ignore_non_existing:
            return

        if len(keys) == 1:
            return_values.append(data[keys[0]])
        else:
            if isinstance(data[keys[0]], list):
                for idx, obj in enumerate(data[keys[0]]):
                    apply_keyed_inplace(obj, keys[1:], return_values)
            else:
                apply_keyed_inplace(data[keys[0]], keys[1:], return_values)

    apply_keyed_inplace(dictionary, keys, return_values)
    return return_values

In [ ]:
test_dict = {'A': [{'B': 'C'}, {'B':'D'}]}
expected = ["C", "D"]
actual = return_keyed(test_dict, ["A", "B"])
assert expected == actual